In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

In [ ]:
# !pip install torch-summary

# **Importing necessary library**

In [ ]:
import torch
import torchvision
import torch.nn as nn                                          # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim                                    # For all Optimization algorithms, SGD, Adam, etc.
import torchvision.transforms as transforms                    # Transformations we can perform on our dataset
from torch.utils.data import (DataLoader,)                     # Gives easier dataset managment and creates mini batches
from torch.utils.tensorboard import SummaryWriter              # to print to tensorboard
import pickle                                                  # for importing dataset from pickle file
import matplotlib.pyplot as plt                                # to plot images
import datetime                                                # for taking the current data and time

# **Defining classs for convolutiona neural network as generator and discriminator**

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, channels_img, features_d):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(
            # torch.nn.ConvTranspose2d(in_channels: int, out_channels: int, kernel_size: Union[T, Tuple[T, T]], stride: Union[T, Tuple[T, T]] = 1, 
            # padding: Union[T, Tuple[T, T]] = 0, output_padding: Union[T, Tuple[T, T]] = 0, groups: int = 1, bias: bool = True, dilation: int = 1, padding_mode: str = 'zeros')
            # N x channels_img x 64 x 64
            # (1, 16, 4, 2, 1)
            nn.Conv2d(channels_img, features_d, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            # N x features_d x 32 x 32
            # (16, 32, 4, 2, 1)
            nn.Conv2d(features_d, features_d * 2, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(features_d * 2),
            nn.LeakyReLU(0.2),
            # (32, 64, 4, 2, 1)
            nn.Conv2d(features_d * 2, features_d * 4, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(features_d * 4),
            nn.LeakyReLU(0.2),
            # (64, 128, 4, 2, 1)
            nn.Conv2d(features_d * 4, features_d * 8, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(features_d * 8),
            nn.LeakyReLU(0.2),
            # N x features_d*8 x 4 x 4
            # (128, 1, 4, 2, 1)
            nn.Conv2d(features_d * 8, 1, kernel_size=4, stride=2, padding=0),
            # N x 1 x 1 x 1
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.net(x)


class Generator(nn.Module):
    def __init__(self, channels_noise, channels_img, features_g):
        super(Generator, self).__init__()

        self.net = nn.Sequential(
            # torch.nn.ConvTranspose2d(in_channels: int, out_channels: int, kernel_size: Union[T, Tuple[T, T]], stride: Union[T, Tuple[T, T]] = 1, 
            # padding: Union[T, Tuple[T, T]] = 0, output_padding: Union[T, Tuple[T, T]] = 0, groups: int = 1, bias: bool = True, dilation: int = 1, padding_mode: str = 'zeros')
            # N x channels_noise x 1 x 1
            # (256, 256, 4, 1, 0)
            nn.ConvTranspose2d(channels_noise, features_g * 16, kernel_size=4, stride=1, padding=0),
            nn.BatchNorm2d(features_g * 16),
            nn.ReLU(),
            # N x features_g*16 x 4 x 4
            # (256, 128, 4, 2, 1)
            nn.ConvTranspose2d(features_g * 16, features_g * 8, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(features_g * 8),
            nn.ReLU(),
            # (128, 64, 4, 2, 1)
            nn.ConvTranspose2d(features_g * 8, features_g * 4, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(features_g * 4),
            nn.ReLU(),
            # (64, 32, 4, 2, 1)
            nn.ConvTranspose2d(features_g * 4, features_g * 2, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(features_g * 2),
            nn.ReLU(),
            # (32, 1, 4, 2, 1)
            nn.ConvTranspose2d(features_g * 2, channels_img, kernel_size=4, stride=2, padding=1),
            # N x channels_img x 64 x 64
            nn.Tanh(),
        )

    def forward(self, x):
        return self.net(x)


        

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


# **Defining the necessary parameter**

In [ ]:

# Hyperparameters
lr = 0.0005
batch_size = 64
image_size = 64
channels_img = 1
channels_noise = 512
num_epochs = 1
number_pixel = 64

# For how many channels Generator and Discriminator should use
features_d = 16
features_g = 16


# **Importing the required dataset**

In [ ]:
pickle_in = open("/content/drive/MyDrive/Colab Notebooks/pytorch/minor project/minor_project_sixth_result_final/Copy of X_desired_shape_resize_64_10000_each_class.pickle","rb")
X = pickle.load(pickle_in)

# **Converting numpy array into tensor**

In [ ]:
X = torch.Tensor(X)
print(X[0].size())
print(type(X))
print(X.shape)

# **Filtering dataset**

In [ ]:
print(X[0].shape)
print(X[:][:][0].shape)

In [ ]:
print(type(X))
print(X.shape)
print(X[0].shape)

In [ ]:
for i in range(0, 460000,2000):
  temp = X[i]
  plt.imshow(temp[:][:].reshape([number_pixel,number_pixel]), cmap='gray')
  if i== 6000:
    break

In [ ]:
X_new = X[0: 360000]
print(X_new.shape)

# **Feature scaling**

In [ ]:
def norm(x_r):
    x_n = (2.0*x_r/255.0 -1)
    return x_n

In [ ]:
def denorm(x_r):
    x_n = 255.0 *(x_r + 1) / 2.0
    return x_n

In [ ]:
for i in range(0, 360000):
  X_new[i] =norm(X_new[i])

In [ ]:
X_new[0,:, 10:15, 10:15]

In [ ]:
torch.min(X_new[0]), torch.max(X_new[0])

In [ ]:
img_norm = denorm(X_new[0])
print(img_norm.shape)
print(img_norm[10:15, 10:15])
plt.imshow(img_norm[:][:].reshape([number_pixel,number_pixel]), cmap='gray')

In [ ]:
dataloader = DataLoader(X_new, batch_size=batch_size, shuffle=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
G_losses = []
D_losses = []

lossG_real_for_plot = []
lossD_real_for_plot = []

In [ ]:
# Create discriminator and generator
netD = Discriminator(channels_img, features_d).to(device)
netG = Generator(channels_noise, channels_img, features_g).to(device)

# Setup Optimizer for G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(0.5, 0.999))

netG.train()
netD.train()

criterion = nn.BCELoss()


real_label = 1
fake_label = 0

fixed_noise = torch.randn(64, channels_noise, 1, 1).to(device)
writer_real = SummaryWriter(f"runs/GAN_NEPALI/test_real")
writer_fake = SummaryWriter(f"runs/GAN_NEPALI/test_fake")
step = 0

In [ ]:
CHECKPOINT_FILENAME = "/content/drive/MyDrive/Colab Notebooks/pytorch/minor project/minor_project_sixth_result_final/checkPoint_holder/my_checkpoint.pth.tar"
def save_checkpoint(state,filename=CHECKPOINT_FILENAME):
  print("********************************Saving check point ******************************")
  torch.save(state,filename)

In [ ]:
def load_checkpoint(checkpoint):
  print("..............Loading the saved checkpoints...........")
  netG.load_state_dict(checkpoint['state_dict_g'])
  optimizerG.load_state_dict(checkpoint['optimizer_g'])
  lossG = checkpoint['loss_g']
  lossD = checkpoint['loss_d']
  D_x = checkpoint['d_out']
  netG.eval()

In [ ]:


print("Starting Training...")

for epoch in range(num_epochs):
    for batch_idx, (data) in enumerate(dataloader):
        data = data.to(device)

        ### Train Discriminator: max log(1 - D(G(z)))
        netD.zero_grad()
        label = (torch.ones(batch_size) * 0.9).to(device)
        output = netD(data).reshape(-1)
        lossD_real = criterion(output, label)
        D_x = output.mean().item()

        noise = torch.randn(batch_size, channels_noise, 1, 1).to(device)
        fake = netG(noise)
        label = (torch.ones(batch_size) * 0.1).to(device)

        output = netD(fake.detach()).reshape(-1)
        lossD_fake = criterion(output, label)

        lossD = lossD_real + lossD_fake
        lossD.backward()
        optimizerD.step()


        ### Train Generator: max log(D(G(z)))
        netG.zero_grad()
        label = torch.ones(batch_size).to(device)
        output = netD(fake).reshape(-1)

        lossG = criterion(output, label)
        lossG.backward()
        optimizerG.step()
        


        # Print losses ocassionally and print to tensorboard
        if batch_idx % 100 == 0:
            step += 1
            print(f"Epoch [{epoch}/{num_epochs}] Batch {batch_idx}/{len(dataloader)} \Loss D: {lossD:.4f}, loss G: {lossG:.4f} D(x): {D_x:.4f}")
            SummaryWriter('runs/gen_loss').add_scalar('generator loss with epoch', lossG/100.0, (epoch*len(dataloader) + batch_idx))
            SummaryWriter('runs/dis_loss').add_scalar('discriminator loss with epoch', lossD/100.0, epoch*len(dataloader) + batch_idx)

            # # to save the model
            # checkpoint = {
            #               'state_dict_g': netG.state_dict(),
            #               'optimizer_g': optimizerG.state_dict(),
            #               'loss_g': lossG,
            #               'loss_d': lossD,
            #               'd_out': D_x
                          
            #               }
            # save_checkpoint(checkpoint)




            # Save Losses for plotting later
            G_losses.append(lossG.item())
            D_losses.append(lossD.item())

            lossG_real_for_plot.append(lossD_real.item())
            lossD_real_for_plot.append(lossD_fake.item())


            with torch.no_grad():
                fake = netG(fixed_noise)
                # torchvision.utils.make_grid(tensor: Union[torch.Tensor, List[torch.Tensor]], nrow: int = 8, padding: int = 2, normalize: bool = False, range: Optional[Tuple[int, int]] = None, scale_each: bool = False, pad_value: int = 0) → torch.Tensor
                img_grid_real = torchvision.utils.make_grid(data[:32], normalize=True)
                img_grid_fake = torchvision.utils.make_grid(fake[:32], normalize=True)
                writer_real.add_image("Nepali letter Real Images", img_grid_real, global_step=step)
                writer_fake.add_image("Nepali letter Fake Images", img_grid_fake, global_step=step)






In [ ]:
print(netD)
print(netG)

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
# plt.figure(figsize=(10,5))
# plt.title("Generator and Discriminator Loss During Training")
# plt.plot(lossG_real_for_plot,label="G_real_score")
# plt.plot(lossD_real_for_plot,label="D_fake_score")
# plt.xlabel("iterations")
# plt.ylabel("score")
# plt.legend()
# plt.title('Score in GAN');
# plt.show()

# **Loading the saved model**

In [ ]:
load_checkpoint(torch.load(CHECKPOINT_FILENAME))


In [ ]:
fake = netG(fixed_noise)
print(fake.shape)

In [ ]:
print(fake[0][0].shape)
print(type(fake[0][0]))

In [ ]:
plt.imshow(fake[20][0].cpu().data.numpy(),  cmap='gray')

In [ ]:
%tensorboard --logdir runs/GAN_NEPALI